In [1]:
import pandas as pd
import time
from ast import literal_eval
import numpy as np

import Filter2 as fil2

from pymongo import MongoClient

#from os import environ
#from sqlalchemy import create_engine
#from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
#from sqlalchemy import Sequence

#from sqlalchemy.dialects.postgresql import array, ARRAY
#import psycopg2
#from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
#import pyodbc

#from sqlalchemy.sql import select

In [2]:
def readCSV(item):
    data = pd.read_csv(item, dtype={0: int, "business_id":str, "stars":np.int8, "date":str, "text":str, "name":str, "postal_code":str, "ccur":np.int64}, index_col=0, encoding='latin-1')
    
    df = pd.DataFrame(data)
    return df

def initialLoad():
    
    allDataLem = readCSV('static/revSamLem2.csv')
    allDataLem["lemmas"] = allDataLem["lemmas"].apply(literal_eval)
    
    return allDataLem

t0 = time.time()
allDataLem = initialLoad()
t1 = time.time()
print("Time is : " + str(t1-t0))

C:\Users\Bogy\Anaconda3\envs\Envir. Python 3.6\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Time is : 363.7233073711395


In [5]:
allDataLem = pd.read_csv('static/stackedData2.csv')

In [8]:
proba0 = allDataLem.copy()
del proba0["Unnamed: 0"]
proba0.dtypes

newID     object
1_star     int64
2_star     int64
3_star     int64
4_star     int64
5_star     int64
dtype: object

In [9]:
#The commented code bellow wil create reviews_mini by removing columns

#proba0.reset_index(inplace = True)
#del proba0["ccur"]
#del proba0["text"]
#del proba0["level_0"]
#proba0["new_name"] = proba0["name"] + " - " +  proba0["postal_code"]
#del proba0["name"]
#del proba0["postal_code"]
#proba0 = proba0[['index', 'business_id', 'new_name', 'date', 'stars', 'lemmas']]
#del proba0["index"]
proba0

,newID,1_star,2_star,3_star,4_star,5_star
0,"""Mon Ami Gabi"" - 89109",234,413,852,2565,3264
1,"""Bacchanal Buffet"" - 89109",667,611,1098,1896,2716
2,"""Wicked Spoon"" - 89109",409,633,1151,1909,1836
3,"""Gordon Ramsay BurGR"" - 89109",315,476,874,1570,2184
4,"""Earl of Sandwich"" - 89109",97,193,597,1643,2453
...,...,...,...,...,...,...
38939,"""Second Cup"" - L6V 4K2",1,0,0,4,6
38940,"""Inferno Gourmet Burger Bar"" - 44256",3,2,2,3,1
38941,"""Baja Fresh Las Vegas"" - 89102",2,1,3,1,4
38942,"""Second Cup"" - H3H 2S7",4,1,3,0,3


In [23]:
pd.to_datetime('2017-08-02', format="%Y-%m-%d")#.dt.date
#np.datetime64('2005-02-25')

Timestamp('2017-08-02 00:00:00')

In [10]:
client = MongoClient()
db = client.test_database
collection = db.test_collection

#posts = db.posts
#posts = db.reviews
#posts = db.reviews_mini
posts = db.restaurants

In [14]:
db.list_collection_names()
#posts.find_one()

['reviews_mini', 'reviews', 'posts', 'restaurants']

In [15]:
#posts.delete_many({})
posts.count_documents({})

38944

In [13]:
t0 = time.time()
posts.insert_many(proba0.to_dict('records'))
t1 = time.time()
(t1-t0)/60

0.014814921220143636

In [31]:
t0 = time.time()
proba0 = allDataLem.copy()
proba1 = fil2.main(proba0, "", '"Mon Ami Gabi" - 89109', "", "", "")
t1 = time.time()
print('pandas filter time: ' + str(t1-t0))
proba1

pandas filter time: 1.8829879760742188


,business_id,stars,date,text,name,postal_code,ccur,lemmas
index,,,,,,,,
441370,4JNXUYY8wbaaDmk3BPzlWw,5,2017-08-02,Dont miss the peppercorn steak . The peppercor...,"""Mon Ami Gabi""",89109,7328,"[do, miss, peppercorn, steak, peppercorn, slig..."
443424,4JNXUYY8wbaaDmk3BPzlWw,3,2013-03-16,This is a great place to come during the summe...,"""Mon Ami Gabi""",89109,7328,"[great, place, come, summer, patio, seat, give..."
444590,4JNXUYY8wbaaDmk3BPzlWw,4,2013-01-04,Mon Ami Gabi is practically a landmark. In fac...,"""Mon Ami Gabi""",89109,7328,"[mon, ami, gabi, practically, landmark, fact, ..."
444589,4JNXUYY8wbaaDmk3BPzlWw,4,2010-07-02,Having done Vegas (at least once) every year f...,"""Mon Ami Gabi""",89109,7328,"[do, vega, little, every, year, past, year, ni..."
444588,4JNXUYY8wbaaDmk3BPzlWw,5,2017-08-04,"To start with, I'm French and I'm a Vegas loca...","""Mon Ami Gabi""",89109,7328,"[start, french, vega, local, time, time, like,..."
...,...,...,...,...,...,...,...,...
443316,4JNXUYY8wbaaDmk3BPzlWw,5,2016-12-12,"amazing, great few of the bella giovanni water...","""Mon Ami Gabi""",89109,7328,"[amaze, great, bella, giovanni, water, show, n..."
443317,4JNXUYY8wbaaDmk3BPzlWw,4,2010-11-18,It sure helps making reservations on a Sunday ...,"""Mon Ami Gabi""",89109,7328,"[sure, help, make, reservation, sunday, night,..."
443314,4JNXUYY8wbaaDmk3BPzlWw,5,2015-04-20,Every time my wife and I have gone to Mon Ami ...,"""Mon Ami Gabi""",89109,7328,"[every, time, wife, go, mon, ami, gabi, enjoya..."


In [32]:
t0 = time.time()
#cursor = posts.find({})
#for document in cursor:
#      print(document)

#n=0
#kolecionercheto = posts.find({"name": '"Mon Ami Gabi"', "postal_code": '89109'})
#for post in kolecionercheto:
    #n = n + 1
    #print(post)
#print(kolecionercheto)
proba2 = pd.DataFrame(list(posts.find({"name": '"Mon Ami Gabi"', "postal_code": '89109'})))
t1 = time.time()
print('pandas filter time: ' + str(t1-t0) + 'for n: ' + str(len(proba2)))
proba2.rename(columns={"_id": "index"}, inplace= True)
proba2.set_index("index", inplace= True)
proba2

pandas filter time: 42.7846794128418for n: 7328


,business_id,stars,date,text,name,postal_code,ccur,lemmas
index,,,,,,,,
5dff854448cc11dacfa82eb3,4JNXUYY8wbaaDmk3BPzlWw,5,2017-08-02,Dont miss the peppercorn steak . The peppercor...,"""Mon Ami Gabi""",89109,7328,"[do, miss, peppercorn, steak, peppercorn, slig..."
5dff854448cc11dacfa82eb4,4JNXUYY8wbaaDmk3BPzlWw,3,2013-03-16,This is a great place to come during the summe...,"""Mon Ami Gabi""",89109,7328,"[great, place, come, summer, patio, seat, give..."
5dff854448cc11dacfa82eb5,4JNXUYY8wbaaDmk3BPzlWw,4,2013-01-04,Mon Ami Gabi is practically a landmark. In fac...,"""Mon Ami Gabi""",89109,7328,"[mon, ami, gabi, practically, landmark, fact, ..."
5dff854448cc11dacfa82eb6,4JNXUYY8wbaaDmk3BPzlWw,4,2010-07-02,Having done Vegas (at least once) every year f...,"""Mon Ami Gabi""",89109,7328,"[do, vega, little, every, year, past, year, ni..."
5dff854448cc11dacfa82eb7,4JNXUYY8wbaaDmk3BPzlWw,5,2017-08-04,"To start with, I'm French and I'm a Vegas loca...","""Mon Ami Gabi""",89109,7328,"[start, french, vega, local, time, time, like,..."
...,...,...,...,...,...,...,...,...
5dff854448cc11dacfa84b4e,4JNXUYY8wbaaDmk3BPzlWw,5,2016-12-12,"amazing, great few of the bella giovanni water...","""Mon Ami Gabi""",89109,7328,"[amaze, great, bella, giovanni, water, show, n..."
5dff854448cc11dacfa84b4f,4JNXUYY8wbaaDmk3BPzlWw,4,2010-11-18,It sure helps making reservations on a Sunday ...,"""Mon Ami Gabi""",89109,7328,"[sure, help, make, reservation, sunday, night,..."
5dff854448cc11dacfa84b50,4JNXUYY8wbaaDmk3BPzlWw,5,2015-04-20,Every time my wife and I have gone to Mon Ami ...,"""Mon Ami Gabi""",89109,7328,"[every, time, wife, go, mon, ami, gabi, enjoya..."


In [87]:
t0 = time.time()
fil2.main(proba1, "", '"Mon Ami Gabi" - 89109', "", "", "")
t1 = time.time()
print('pandas filter time: ' + str(t1-t0))
proba1.dtypes

pandas filter time: 0.013936281204223633


business_id    object
stars            int8
date           object
text           object
name           object
postal_code    object
ccur            int64
lemmas         object
dtype: object

In [82]:
t0 = time.time()
fil2.main(proba2, "", '"Mon Ami Gabi" - 89109', "", "", "")
t1 = time.time()
print('pandas filter time: ' + str(t1-t0))
proba2.dtypes

pandas filter time: 0.028954744338989258


business_id    object
stars           int64
date           object
text           object
name           object
postal_code    object
ccur            int64
lemmas         object
dtype: object

In [92]:
t0 = time.time()
fil2.main(proba3, "", '"Mon Ami Gabi" - 89109', "", "", "")
t1 = time.time()
print('pandas filter time: ' + str(t1-t0))
proba3.dtypes

pandas filter time: 0.012949705123901367


business_id    object
stars            int8
date           object
text           object
name           object
postal_code    object
ccur            int64
lemmas         object
dtype: object

In [40]:
proba3 = proba2.copy()
proba3.reset_index(inplace=True)
proba3.drop(columns=['index'], inplace=True)
proba3

,business_id,stars,date,text,name,postal_code,ccur,lemmas
0,4JNXUYY8wbaaDmk3BPzlWw,5,2017-08-02,Dont miss the peppercorn steak . The peppercor...,"""Mon Ami Gabi""",89109,7328,"[do, miss, peppercorn, steak, peppercorn, slig..."
1,4JNXUYY8wbaaDmk3BPzlWw,3,2013-03-16,This is a great place to come during the summe...,"""Mon Ami Gabi""",89109,7328,"[great, place, come, summer, patio, seat, give..."
2,4JNXUYY8wbaaDmk3BPzlWw,4,2013-01-04,Mon Ami Gabi is practically a landmark. In fac...,"""Mon Ami Gabi""",89109,7328,"[mon, ami, gabi, practically, landmark, fact, ..."
3,4JNXUYY8wbaaDmk3BPzlWw,4,2010-07-02,Having done Vegas (at least once) every year f...,"""Mon Ami Gabi""",89109,7328,"[do, vega, little, every, year, past, year, ni..."
4,4JNXUYY8wbaaDmk3BPzlWw,5,2017-08-04,"To start with, I'm French and I'm a Vegas loca...","""Mon Ami Gabi""",89109,7328,"[start, french, vega, local, time, time, like,..."
...,...,...,...,...,...,...,...,...
7323,4JNXUYY8wbaaDmk3BPzlWw,5,2016-12-12,"amazing, great few of the bella giovanni water...","""Mon Ami Gabi""",89109,7328,"[amaze, great, bella, giovanni, water, show, n..."
7324,4JNXUYY8wbaaDmk3BPzlWw,4,2010-11-18,It sure helps making reservations on a Sunday ...,"""Mon Ami Gabi""",89109,7328,"[sure, help, make, reservation, sunday, night,..."
7325,4JNXUYY8wbaaDmk3BPzlWw,5,2015-04-20,Every time my wife and I have gone to Mon Ami ...,"""Mon Ami Gabi""",89109,7328,"[every, time, wife, go, mon, ami, gabi, enjoya..."
7326,4JNXUYY8wbaaDmk3BPzlWw,4,2009-06-24,Wonderful experience. We decided to go to dinn...,"""Mon Ami Gabi""",89109,7328,"[wonderful, experience, decide, go, dinner, la..."


In [73]:
proba3.equals(proba1)

False

In [72]:
proba3.reset_index(inplace=True)
proba3.set_index(["index"], inplace=True)
proba3

,business_id,stars,date,text,name,postal_code,ccur,lemmas
index,,,,,,,,
0,4JNXUYY8wbaaDmk3BPzlWw,5,2017-08-02,Dont miss the peppercorn steak . The peppercor...,"""Mon Ami Gabi""",89109,7328,"[do, miss, peppercorn, steak, peppercorn, slig..."
1,4JNXUYY8wbaaDmk3BPzlWw,3,2013-03-16,This is a great place to come during the summe...,"""Mon Ami Gabi""",89109,7328,"[great, place, come, summer, patio, seat, give..."
2,4JNXUYY8wbaaDmk3BPzlWw,4,2013-01-04,Mon Ami Gabi is practically a landmark. In fac...,"""Mon Ami Gabi""",89109,7328,"[mon, ami, gabi, practically, landmark, fact, ..."
3,4JNXUYY8wbaaDmk3BPzlWw,4,2010-07-02,Having done Vegas (at least once) every year f...,"""Mon Ami Gabi""",89109,7328,"[do, vega, little, every, year, past, year, ni..."
4,4JNXUYY8wbaaDmk3BPzlWw,5,2017-08-04,"To start with, I'm French and I'm a Vegas loca...","""Mon Ami Gabi""",89109,7328,"[start, french, vega, local, time, time, like,..."
...,...,...,...,...,...,...,...,...
7323,4JNXUYY8wbaaDmk3BPzlWw,5,2016-12-12,"amazing, great few of the bella giovanni water...","""Mon Ami Gabi""",89109,7328,"[amaze, great, bella, giovanni, water, show, n..."
7324,4JNXUYY8wbaaDmk3BPzlWw,4,2010-11-18,It sure helps making reservations on a Sunday ...,"""Mon Ami Gabi""",89109,7328,"[sure, help, make, reservation, sunday, night,..."
7325,4JNXUYY8wbaaDmk3BPzlWw,5,2015-04-20,Every time my wife and I have gone to Mon Ami ...,"""Mon Ami Gabi""",89109,7328,"[every, time, wife, go, mon, ami, gabi, enjoya..."


In [79]:
proba3 = proba3.astype({'stars': np.int8})
proba3.equals(proba1)
proba3.dtypes

business_id    object
stars            int8
date           object
text           object
name           object
postal_code    object
ccur            int64
lemmas         object
dtype: object